In [13]:

from bs4 import BeautifulSoup
import requests
import pandas as pd
 
import numpy as np
import time
import folium
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors


!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values



Solving environment: ...working... done

# All requested packages already installed.



In [14]:
url='https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Hyderabad,_India'
country = 'India',
city = 'Hyderabad'
geolocator = Nominatim()
page = requests.get(url)
parsedHTML = BeautifulSoup(page.text, 'html.parser')

C:\Users\justin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


In [15]:
citiesContainers = parsedHTML.find_all('div', class_='mw-category-group')
cities = [];
c = 1;
for citiesContainer in citiesContainers:
    for li in citiesContainer.find_all('li'):
        cities.append(li.text.strip().split(',')[0])
print(len(cities))

200


**collected all cities list**

In [16]:
cities_df = pd.DataFrame({'Neighbourhoods': cities})
cities_df.head(10)

,Neighbourhoods
0,A. S. Rao Nagar
1,A.C. Guards
2,Abhyudaya Nagar
3,Abids
4,Adikmet
5,Afzal Gunj
6,Aghapura
7,Aliabad
8,Alijah Kotla
9,Allwyn Colony


**collect all geo coordiantes for each neighnourhood**

In [17]:
latitudes = []
longitudes = []
errorCities = []
for neighbourhood in cities:
    print(neighbourhood)
    address = "{},{},{}".format(neighbourhood, city, country)
    retries = 3
    while (retries > 0):
        try:
            location = geolocator.geocode(address)
            latitude = location.latitude
            longitude = location.longitude
            latitudes.append(latitude)
            longitudes.append(longitude)
            retries = 0
            if neighbourhood in errorCities:
                errorCities.remove(neighbourhood)
        except Exception as e:
            print("ERROR: ", address, e)
            if not neighbourhood in errorCities:
                errorCities.append(neighbourhood)
            retries = retries - 1
            time.sleep(1)

A. S. Rao Nagar
A.C. Guards
Abhyudaya Nagar
Abids
Adikmet
Afzal Gunj
ERROR:  Afzal Gunj,Hyderabad,('India',) 'NoneType' object has no attribute 'latitude'
ERROR:  Afzal Gunj,Hyderabad,('India',) 'NoneType' object has no attribute 'latitude'
ERROR:  Afzal Gunj,Hyderabad,('India',) 'NoneType' object has no attribute 'latitude'
Aghapura
Aliabad
Alijah Kotla
Allwyn Colony
Alwal
Amberpet
Ameerpet
Ashok Nagar
Asif Nagar
Attapur
Azamabad
Azampura
Badichowdi
Bagh Lingampally
Bairamalguda
Balkampet
Banjara Hills
Bank Street
Barkas
Barkatpura
Basheerbagh
Bazarghat
Begum Bazaar
ERROR:  Begum Bazaar,Hyderabad,('India',) Service timed out
Begumpet
Bharat Nagar
BHEL Township
BJR Nagar
ERROR:  BJR Nagar,Hyderabad,('India',) 'NoneType' object has no attribute 'latitude'
ERROR:  BJR Nagar,Hyderabad,('India',) 'NoneType' object has no attribute 'latitude'
ERROR:  BJR Nagar,Hyderabad,('India',) 'NoneType' object has no attribute 'latitude'
Boggulkunta
ERROR:  Boggulkunta,Hyderabad,('India',) 'NoneType' o

In [18]:
cities_with_geo_coordinates = filter(lambda city: not city in errorCities, cities)
cities_with_geo_coordinates = [city for city in cities_with_geo_coordinates]

In [20]:
cities_with_geo_coordinates[0:10]

['A. S. Rao Nagar',
 'A.C. Guards',
 'Abhyudaya Nagar',
 'Abids',
 'Adikmet',
 'Aghapura',
 'Aliabad',
 'Alijah Kotla',
 'Allwyn Colony',
 'Alwal']

**prepare a dataframe with required features**

In [21]:
cities_df = pd.DataFrame({'Neighbourhood': cities_with_geo_coordinates, 'Latitude': latitudes, 'Longitude': longitudes})

In [22]:
cities_df.head(10)

,Neighbourhood,Latitude,Longitude
0,A. S. Rao Nagar,17.479950,78.556834
1,A.C. Guards,17.402804,78.459487
2,Abhyudaya Nagar,17.337661,78.564716
3,Abids,17.389478,78.477182
4,Adikmet,17.409550,78.513094
5,Aghapura,17.389178,78.465273
6,Aliabad,17.345630,78.472680
7,Alijah Kotla,17.360545,78.480102
8,Allwyn Colony,17.504362,78.414985
9,Alwal,17.502229,78.508858


In [23]:
cities_df.tail(10)

,Neighbourhood,Latitude,Longitude
161,Secunderabad,17.469089,78.505918
162,Serilingampally,17.465657,78.340672
163,Shahran Market,17.359578,78.474090
164,Shanker Mutt,17.400764,78.507340
165,Shivam Road,17.336795,78.602895
166,Sikh Village,17.460098,78.487287
167,Silpa Avenue Colony,17.439550,78.367562
168,Sindhi Colony,17.441219,78.481048
169,Sitaphalmandi,17.429733,78.517426
170,Somajiguda,17.425505,78.458520


In [24]:
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(city, latitude, longitude))

The geograpical coordinate of Hyderabad are 17.38878595, 78.46106473453146.


**View map of all neighbourhoods**

In [26]:

hyderabad_map = folium.Map(location=[latitude, longitude], zoom_start=10)
for neighbourhood, latiude, longitude in zip(cities_df['Neighbourhood'], cities_df['Latitude'], cities_df['Longitude']):
    label = neighbourhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latiude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(hyderabad_map) 
hyderabad_map

In [27]:
LIMIT = 100
radius = 500

CLIENT_ID = 'ZPT23A2KX3YWPVWSILJD5ABOSZIM241NTZU3PWVDWTMRWVFO'
CLIENT_SECRET = 'C25N1NKBSJUWOXIAGSHR1WB4135VYYH4TK3LEOB4S2JAX0OQ'
VERSION = '20180605'

**Use folium to get venues of each neighbourhoods**

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
hyderabad_venues = getNearbyVenues(names=cities_df['Neighbourhood'],
                                   latitudes=cities_df['Latitude'],
                                   longitudes=cities_df['Longitude'],
                                  )

In [31]:

print(hyderabad_venues.shape)
hyderabad_venues.head()

(1154, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,A. S. Rao Nagar,17.47995,78.556834,Cafe Coffee Day,17.481262,78.555077,Café
1,A. S. Rao Nagar,17.47995,78.556834,HDFC Bank,17.480961,78.555580,Bank
2,A. S. Rao Nagar,17.47995,78.556834,Max,17.478846,78.558801,Clothing Store
3,A. S. Rao Nagar,17.47995,78.556834,Ratnadeep Supermarket,17.481483,78.554160,Department Store
4,A. S. Rao Nagar,17.47995,78.556834,Swagath Grand,17.482022,78.553261,Indian Restaurant


In [32]:

hyderabad_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
A. S. Rao Nagar,8,8,8,8,8,8
A.C. Guards,12,12,12,12,12,12
Abhyudaya Nagar,7,7,7,7,7,7
Abids,18,18,18,18,18,18
Adikmet,4,4,4,4,4,4
...,...,...,...,...,...,...
Sikh Village,6,6,6,6,6,6
Silpa Avenue Colony,6,6,6,6,6,6
Sindhi Colony,13,13,13,13,13,13


In [33]:

print('There are {} uniques categories.'.format(len(hyderabad_venues['Venue Category'].unique())))

There are 154 uniques categories.


**One hot enconding for categorial values**

In [34]:
# one hot encoding
hyderabad_onehot = pd.get_dummies(hyderabad_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hyderabad_onehot['Neighborhood'] = hyderabad_venues['Neighborhood'] 
hyderabad_onehot.head()

,ATM,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Amphitheater,Andhra Restaurant,Antique Shop,Arcade,Arts & Crafts Store,...,Supermarket,Szechuan Restaurant,Tea Room,Thai Restaurant,Theme Park,Toy / Game Store,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:

hyderabad_grouped = hyderabad_onehot.groupby('Neighborhood').mean().reset_index()
print(hyderabad_grouped.shape)
hyderabad_grouped.head()

(160, 154)


,Neighborhood,ATM,Accessories Store,Afghan Restaurant,Airport Terminal,American Restaurant,Amphitheater,Andhra Restaurant,Antique Shop,Arcade,...,Supermarket,Szechuan Restaurant,Tea Room,Thai Restaurant,Theme Park,Toy / Game Store,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Women's Store
0,A. S. Rao Nagar,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A.C. Guards,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Abhyudaya Nagar,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Abids,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Adikmet,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
num_top_venues = 10

for hood in hyderabad_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = hyderabad_grouped[hyderabad_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----A. S. Rao Nagar----
                     venue  freq
0         Department Store  0.25
1           Clothing Store  0.12
2        Indian Restaurant  0.12
3                     Bank  0.12
4                    Diner  0.12
5                     Café  0.12
6       Chinese Restaurant  0.12
7             Night Market  0.00
8  New American Restaurant  0.00
9              Music Venue  0.00


----A.C. Guards----
                       venue  freq
0          Indian Restaurant  0.25
1      Hyderabadi Restaurant  0.17
2                      Hotel  0.08
3  Middle Eastern Restaurant  0.08
4             Ice Cream Shop  0.08
5    South Indian Restaurant  0.08
6                       Café  0.08
7           Department Store  0.08
8                     Bakery  0.08
9                        ATM  0.00


----Abhyudaya Nagar----
                       venue  freq
0                      Hotel  0.29
1          Indian Restaurant  0.29
2               Amphitheater  0.14
3               Burger Joint  0.14
4    

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

**Top 10 venues**

In [38]:
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{}'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = hyderabad_grouped['Neighborhood']

for ind in np.arange(hyderabad_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hyderabad_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(8)

,Neighborhood,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
0,A. S. Rao Nagar,Department Store,Chinese Restaurant,Café,Bank,Diner,Indian Restaurant,Clothing Store,Convenience Store,Cosmetics Shop,Cricket Ground
1,A.C. Guards,Indian Restaurant,Hyderabadi Restaurant,South Indian Restaurant,Department Store,Middle Eastern Restaurant,Hotel,Café,Bakery,Ice Cream Shop,Women's Store
2,Abhyudaya Nagar,Indian Restaurant,Hotel,Amphitheater,Burger Joint,Food Truck,Women's Store,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop
3,Abids,Indian Restaurant,Juice Bar,Hotel,Electronics Store,Diner,Shoe Store,Mobile Phone Shop,Shopping Mall,Fast Food Restaurant,Snack Place
4,Adikmet,Clothing Store,Cosmetics Shop,Burger Joint,Ice Cream Shop,Comedy Club,Concert Hall,Convenience Store,Coffee Shop,Cricket Ground,Cupcake Shop
5,Aghapura,Indian Restaurant,Afghan Restaurant,Bakery,Platform,Women's Store,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Department Store
6,Aliabad,Gym,Sandwich Place,Smoke Shop,Snack Place,Ice Cream Shop,Asian Restaurant,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop
7,Alijah Kotla,Cheese Shop,Cosmetics Shop,Cupcake Shop,Electronics Store,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Department Store,Deli / Bodega


**Get 5 clusters**

In [39]:
kclusters = 5

hyderabad_grouped_clustering = hyderabad_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hyderabad_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 2, 2, 2, 1, 2, 1, 1, 0, 1])

In [40]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

hyderabad_merged = cities_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
hyderabad_merged = hyderabad_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
hyderabad_merged.dropna(subset=['Cluster Labels'], inplace=True)
hyderabad_merged.head(8) # check the last columns!

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
0,A. S. Rao Nagar,17.479950,78.556834,1.0,Department Store,Chinese Restaurant,Café,Bank,Diner,Indian Restaurant,Clothing Store,Convenience Store,Cosmetics Shop,Cricket Ground
1,A.C. Guards,17.402804,78.459487,2.0,Indian Restaurant,Hyderabadi Restaurant,South Indian Restaurant,Department Store,Middle Eastern Restaurant,Hotel,Café,Bakery,Ice Cream Shop,Women's Store
2,Abhyudaya Nagar,17.337661,78.564716,2.0,Indian Restaurant,Hotel,Amphitheater,Burger Joint,Food Truck,Women's Store,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop
3,Abids,17.389478,78.477182,2.0,Indian Restaurant,Juice Bar,Hotel,Electronics Store,Diner,Shoe Store,Mobile Phone Shop,Shopping Mall,Fast Food Restaurant,Snack Place
4,Adikmet,17.409550,78.513094,1.0,Clothing Store,Cosmetics Shop,Burger Joint,Ice Cream Shop,Comedy Club,Concert Hall,Convenience Store,Coffee Shop,Cricket Ground,Cupcake Shop
5,Aghapura,17.389178,78.465273,2.0,Indian Restaurant,Afghan Restaurant,Bakery,Platform,Women's Store,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Department Store
6,Aliabad,17.345630,78.472680,1.0,Gym,Sandwich Place,Smoke Shop,Snack Place,Ice Cream Shop,Asian Restaurant,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop
7,Alijah Kotla,17.360545,78.480102,1.0,Cheese Shop,Cosmetics Shop,Cupcake Shop,Electronics Store,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Department Store,Deli / Bodega


In [41]:
hyderabad_merged['Cluster Labels'] = hyderabad_merged['Cluster Labels'].fillna(0.0)

In [42]:

hyderabad_merged['Cluster Labels'] = hyderabad_merged['Cluster Labels'].map(lambda x: int(x))

In [43]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hyderabad_merged['Latitude'], hyderabad_merged['Longitude'], hyderabad_merged['Neighbourhood'], hyderabad_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Cluster Analysis**

**Cluster 1**

In [44]:
hyderabad_merged.loc[hyderabad_merged['Cluster Labels'] == 0, hyderabad_merged.columns[[0, 1, 2] + list(range(4, hyderabad_merged.shape[1]))]]

,Neighbourhood,Latitude,Longitude,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
8,Allwyn Colony,17.504362,78.414985,ATM,Cupcake Shop,Electronics Store,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Department Store,Deli / Bodega,Cricket Ground
31,Borabanda,17.451670,78.415421,ATM,Movie Theater,Cupcake Shop,Electronics Store,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Department Store,Deli / Bodega
70,Kamala Nagar,17.328264,78.577015,ATM,Movie Theater,Jewelry Store,Fruit & Vegetable Store,Cupcake Shop,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Department Store
73,Karmanghat,17.341006,78.532975,ATM,Pharmacy,Athletics & Sports,Deli / Bodega,Electronics Store,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Department Store
137,Patancheru,17.528609,78.267425,ATM,Music Venue,Cupcake Shop,Electronics Store,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Department Store,Deli / Bodega
155,Saidabad,17.359533,78.514798,ATM,Coffee Shop,Cupcake Shop,Electronics Store,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Department Store,Deli / Bodega
158,Sanathnagar,17.456965,78.443478,ATM,Department Store,Cupcake Shop,Electronics Store,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Deli / Bodega,Cricket Ground


**Cluster 2**

In [45]:

hyderabad_merged.loc[hyderabad_merged['Cluster Labels'] == 1, hyderabad_merged.columns[[0, 1, 2] + list(range(4, hyderabad_merged.shape[1]))]]

,Neighbourhood,Latitude,Longitude,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
0,A. S. Rao Nagar,17.479950,78.556834,Department Store,Chinese Restaurant,Café,Bank,Diner,Indian Restaurant,Clothing Store,Convenience Store,Cosmetics Shop,Cricket Ground
4,Adikmet,17.409550,78.513094,Clothing Store,Cosmetics Shop,Burger Joint,Ice Cream Shop,Comedy Club,Concert Hall,Convenience Store,Coffee Shop,Cricket Ground,Cupcake Shop
6,Aliabad,17.345630,78.472680,Gym,Sandwich Place,Smoke Shop,Snack Place,Ice Cream Shop,Asian Restaurant,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop
7,Alijah Kotla,17.360545,78.480102,Cheese Shop,Cosmetics Shop,Cupcake Shop,Electronics Store,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Department Store,Deli / Bodega
9,Alwal,17.502229,78.508858,Gym,Grocery Store,Park,Bakery,Cupcake Shop,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Department Store
...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,Serilingampally,17.465657,78.340672,Grocery Store,Bakery,Pharmacy,Restaurant,Cupcake Shop,Donut Shop,Diner,Dessert Shop,Department Store,Deli / Bodega
163,Shahran Market,17.359578,78.474090,Asian Restaurant,Indian Restaurant,Coffee Shop,History Museum,Clothing Store,Juice Bar,Café,Dessert Shop,Monument / Landmark,Donut Shop
166,Sikh Village,17.460098,78.487287,Grocery Store,Indian Restaurant,Airport Terminal,Middle Eastern Restaurant,Café,Business Service,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop
168,Sindhi Colony,17.441219,78.481048,Ice Cream Shop,Indian Restaurant,Pizza Place,Chinese Restaurant,Smoke Shop,Food,Bakery,Sandwich Place,Café,Coffee Shop


**Cluster 3**

In [46]:
hyderabad_merged.loc[hyderabad_merged['Cluster Labels'] == 2, hyderabad_merged.columns[[0, 1, 2] + list(range(4, hyderabad_merged.shape[1]))]]

,Neighbourhood,Latitude,Longitude,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
1,A.C. Guards,17.402804,78.459487,Indian Restaurant,Hyderabadi Restaurant,South Indian Restaurant,Department Store,Middle Eastern Restaurant,Hotel,Café,Bakery,Ice Cream Shop,Women's Store
2,Abhyudaya Nagar,17.337661,78.564716,Indian Restaurant,Hotel,Amphitheater,Burger Joint,Food Truck,Women's Store,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop
3,Abids,17.389478,78.477182,Indian Restaurant,Juice Bar,Hotel,Electronics Store,Diner,Shoe Store,Mobile Phone Shop,Shopping Mall,Fast Food Restaurant,Snack Place
5,Aghapura,17.389178,78.465273,Indian Restaurant,Afghan Restaurant,Bakery,Platform,Women's Store,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Department Store
10,Amberpet,17.390263,78.516481,Pharmacy,Movie Theater,Indian Restaurant,Women's Store,Cupcake Shop,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Department Store
11,Ameerpet,17.437501,78.448251,Indian Restaurant,Department Store,Buffet,Coffee Shop,Vegetarian / Vegan Restaurant,Candy Store,Restaurant,Hotel,Café,Supermarket
16,Azampura,17.376193,78.490874,Indian Restaurant,Hotel,Women's Store,Electronics Store,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Department Store,Deli / Bodega
20,Balkampet,17.446923,78.450451,Indian Restaurant,Train Station,Hockey Arena,Bakery,Light Rail Station,Women's Store,Electronics Store,Dumpling Restaurant,Donut Shop,Diner
22,Bank Street,17.385717,78.480157,Indian Restaurant,Juice Bar,Bookstore,Chaat Place,Snack Place,South Indian Restaurant,Department Store,Dumpling Restaurant,Donut Shop,Diner
27,Begum Bazaar,17.373079,78.470555,Women's Store,Indian Restaurant,Food,Ice Cream Shop,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Cupcake Shop


**Cluster 4**

In [47]:

hyderabad_merged.loc[hyderabad_merged['Cluster Labels'] == 3, hyderabad_merged.columns[[0, 1, 2] + list(range(4, hyderabad_merged.shape[1]))]]

,Neighbourhood,Latitude,Longitude,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
58,Hasmathpet,17.476170,78.482477,Café,Furniture / Home Store,Women's Store,Cupcake Shop,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Department Store,Deli / Bodega
74,Karwan,17.380082,78.437475,Café,Fried Chicken Joint,Women's Store,Cupcake Shop,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Department Store,Deli / Bodega
95,Madannapet,17.358558,78.497106,Café,Gym,Pharmacy,Cupcake Shop,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Department Store,Deli / Bodega
106,Manovikas Nagar,17.468330,78.485153,Café,Garden,Gaming Cafe,Pizza Place,Women's Store,Cricket Ground,Donut Shop,Diner,Dessert Shop,Department Store
117,Moula-Ali,17.466816,78.559216,Boarding House,Café,Cupcake Shop,Electronics Store,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Department Store,Deli / Bodega
119,Mylargadda,17.422291,78.516970,Café,Women's Store,Cupcake Shop,Electronics Store,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Department Store,Deli / Bodega
125,Namalagundu,17.422247,78.516926,Café,Women's Store,Cupcake Shop,Electronics Store,Dumpling Restaurant,Donut Shop,Diner,Dessert Shop,Department Store,Deli / Bodega


**Cluster 5**

In [48]:
hyderabad_merged.loc[hyderabad_merged['Cluster Labels'] == 4, hyderabad_merged.columns[[0, 1, 2] + list(range(4, hyderabad_merged.shape[1]))]]

,Neighbourhood,Latitude,Longitude,1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th
65,Jahanuma,17.343352,78.461464,Diner,Women's Store,Cupcake Shop,Electronics Store,Dumpling Restaurant,Donut Shop,Dessert Shop,Department Store,Deli / Bodega,Cricket Ground
103,Mallapur,17.440537,78.578855,Diner,Women's Store,Cupcake Shop,Electronics Store,Dumpling Restaurant,Donut Shop,Dessert Shop,Department Store,Deli / Bodega,Cricket Ground
130,Nawab Saheb Kunta,17.339792,78.457836,ATM,Diner,Cupcake Shop,Electronics Store,Dumpling Restaurant,Donut Shop,Dessert Shop,Department Store,Deli / Bodega,Cricket Ground
